# Dynamical Structure Factor

## Estimate the minimum nev needed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from mpl_toolkits.mplot3d import axes3d
from matplotlib.collections import PolyCollection
#%matplotlib notebook

### Parameters

In [ ]:
dim      = 2            # Dimension of the lattice
nspins   = 32           # Number of spins in the system

if dim == 1:
    Lx   = 1            # Number of spins in the system in x direction
    Ly   = nspins       # Number of spins in the system in y direction
else:
    Lx   = 4
    Ly   = 8
    
rep      = 1            # Disorder realizations
MAX_nev  = 150          # Maximum number of nev in the file
sizes    = 3
nev      = [50,100,150]
#nev      = [50,100,106,110,1000]

#q_idx    = [i for i in range(24)] #Indices of q to take
q_idx    = [0,1,2,3,4,8,9,10,11,12,16,17,18,19,20]
q_idxX   = [i//Ly for i in q_idx]
q_idxY   = [i%Ly for i in q_idx]
print("q:", q_idx)
print("qx:", q_idxX)
print("qy:", q_idxY)
nQ       = len(q_idx)   # Number of q values to analyse

J1       = 1.0          # Nearest neighbours coupling
D1       = 1.0          # Nearest neighbours anisotropy
J2       = 1.0          # Next-nearest neighbours coupling
D2       = 1.0          # Next-nearest neighbours anisotropy

nW       = 5001         # Number of frequency values to plot
w_min    = 0            # Minimum value of frequency to plot
w_max    = 5            # Maximum value of frequency to plot
epsilon  = 1.0/nspins   # Constant to compute the Dirac delta function

#path2INdata  = 
#path2OUTdata = 
path2INdata  = 
path2OUTdata = 

### Prepare data

#### 1. Read data from files

In [ ]:
# DeltaEn and factorN have the values of (En-E0) and the preceeding factor of the DSF
DeltaEn = np.empty((MAX_nev*nQ,rep))
factorN = np.empty((MAX_nev*nQ,rep))

# Loop over disorder realizations
for dis_rep in range(rep):
    
    # Loop over excited states
    with open(path2INdata + 'dsf_d' + str(dis_rep) + '_' + str(nspins) + '_' + str(MAX_nev) + '.dat') as INfile:
        array = np.genfromtxt(INfile)
        tmp_DeltaEn, tmp_factorN = array.T
            
    
    DeltaEn[:,dis_rep] = tmp_DeltaEn
    factorN[:,dis_rep] = tmp_factorN

print(DeltaEn.shape)
print(factorN.shape)

#### 2. Create the frequency-dependent variables

In [ ]:
delta_w  = (w_max - w_min)/(nW - 1)
w        = np.linspace(w_min,w_max,nW)
Sw       = np.zeros((nW,nQ,sizes))

# Funtion to compute the Dirac delta
pi     = np.arccos(-1)
inv_pi = 1.0/pi

def DiracDelta(w,deltaEn,e2):
    return inv_pi*e2/(e2 + (w-deltaEn)*(w-deltaEn))

# Loop over q values
e2 = epsilon*epsilon

# Loop over disorder realizations
for dis_rep in range(rep):  
    
    # Loop over q values
    tmp_Sw = np.zeros((rep,nW,nQ,sizes))
    for wi in range(nW):
        
        for q in range(nQ):
            for s in range(sizes):
                start_off = MAX_nev * q + 1
                end_off   = MAX_nev * q + nev[s]
                mid_array = factorN[start_off:end_off,dis_rep] * DiracDelta(w[wi],DeltaEn[start_off:end_off,dis_rep],e2)
                tmp_Sw[dis_rep,wi,q,s] = np.sum(mid_array)
                
                #print("q: ", q, "start: ", start_off, "end: ", end_off)

# Compute average over disorder realizations
Sw = np.sum(tmp_Sw, axis=0)
Sw *= 2*pi
Sw /= rep

print(Sw.shape)

### Plot

#### 1. Plot for every q separately

In [ ]:
# Plot parameters
subsX = nQ//4
subsY = 4
if subsX*subsY != nQ:
    subsX += 1

fig, ax = plt.subplots(subsX, subsY, gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(20,20))
fig.suptitle("Dynamical Structure Factor for different q values", fontsize=30)


for i in range(subsX):
    for j in range(subsY):
        qij = i*subsY + j
        #print(qij,q_idx[qij])
        if qij >= nQ:
            break
        ax[i,j].set_title('q: ' + str(q_idx[qij]) + '--> q = (' + str(q_idxX[qij]) + ',' + str(q_idxY[qij]) + ')')
        ax[i,j].set_xlabel('w')
        ax[i,j].set_ylabel('S(w)')
        ax[i,j].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax[i,j].grid()
        ax[i,j].plot(w,Sw[:,qij,0:2])

In [ ]:
if dim == 2 and nspins == 32:
    fig3, ax3 = plt.subplots(1,2,gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(20,5))

    ax3[0].set_title('S(w) for q=20 --> LARGER peak')
    ax3[0].set_xlabel('w')
    ax3[0].set_ylabel('S(w)')
    ax3[0].grid(which='both')
    ax3[0].plot(w, Sw[:,14,])
    
    ax3[1].set_title('S(w) for q=20 --> SMALLER peak')
    ax3[1].set_xlabel('w')
    ax3[1].set_ylabel('S(w)')
    ax3[1].grid(which='both')
    ax3[1].plot(w[1000:4000], Sw[1000:4000,14,2])

In [ ]:
if dim == 2 and nspins == 16:
    fig3, ax3 = plt.subplots(1,2,gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(10,5))

    ax3[0].set_title('S(w) for q=10 --> LARGER peak')
    ax3[0].set_xlabel('w')
    ax3[0].set_ylabel('S(w)')
    ax3[0].grid()
    ax3[0].plot(w, Sw[:,10,2:4])

    ax3[1].set_title('S(w) for q=10 --> SMALLER peak')
    ax3[1].set_xlabel('w')
    ax3[1].set_ylabel('S(w)')
    ax3[1].grid()
    ax3[1].plot(w[3000:5000],Sw[3000:5000,10,2:4])